In [ ]:
# codigo de scrapeo (que sirve tanto para la primera vez que se scrapea como para el resto de veces. va guardando csv con nombres propios para que se guarden en csv diferentes cada vez distinta que se scrapea)

from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time, json, re
import numpy as np
import pandas as pd
from datetime import date

# Config inicial
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")

# Driver principal
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
driver.set_script_timeout(100)

# Botón "Ver más"
ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

# Scroll profundo
last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(200):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))
    print(f"[Principal Scroll {i}] Scroll height: {new_height} - Productos: {productos}")
    if new_height == last_height and i > 5:
        break
    last_height = new_height

# Recolectar URLs de producto
productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))
productos_urls = []
for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    if data_tcproduct:
        product_data = json.loads(data_tcproduct)
        product_url = product_data.get('product_url_page')
        product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
        productos_urls.append(product_url)

productos_urls = productos_urls[:5]
print(f"\n🟢 Productos en página principal: {len(productos_urls)}")
driver.quit()

# Filtros
palabras_clave = ["formats", "responsibleBeauty", "eyeshadowEffects", "lipEffects", "mascaraEffects", "typesHairBrushes", "formulations", "skinTypes", "covers", "finishes", "texture"]
mapa_filtros = {
    "formats": "formato",
    "responsibleBeauty": "responsabilidad",
    "eyeshadowEffects": "efecto_sombra",
    "lipEffects": "efecto_labios",
    "mascaraEffects": "efecto_mascara",
    "typesHairBrushes": "tipo_brocha",
    "formulations": "formulacion",
    "skinTypes": "tipo_piel",
    "covers": "cobertura",
    "finishes": "acabado",
    "texture": "textura"
}

filtros_urls = []
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

filtros = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "gtmrefinement.empty.refinement-item")))
for f in filtros:
    href = f.get_attribute("href")
    if href and any(p in href for p in palabras_clave):
        filtros_urls.append(href)

driver.quit()

# Scrapeo de filtros
productos_por_filtro = {col: [] for col in mapa_filtros.values()}
driver = webdriver.Chrome(service=service, options=options)

for filtro_url in filtros_urls:
    driver.get(filtro_url)
    time.sleep(3)

    try:
        ver_mas_btn = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
        time.sleep(2)
        driver.execute_script("arguments[0].click();", ver_mas_btn)
        print("🔘 Botón 'Ver más' clicado en filtro.")
    except:
        print("ℹ️ No hay botón 'Ver más' en este filtro.")

    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(200):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height and i > 5:
            break
        last_height = new_height

    try:
        valor = driver.find_element(By.CLASS_NAME, "breadcrumb-refinement-value").text.strip()
    except:
        valor = "Valor desconocido"

    productos_en_filtro = []
    productos = driver.find_elements(By.CLASS_NAME, "product-tile.clickable.omnibus-tile")
    for p in productos:
        data = p.get_attribute('data-tcproduct')
        if data:
            prod_data = json.loads(data)
            url_producto = prod_data.get('product_url_page')
            if url_producto:
                url_producto = re.sub(r'-p(\d+)\.html$', r'-P\1.html', url_producto)
                productos_en_filtro.append(url_producto)

    for clave, columna in mapa_filtros.items():
        if clave in filtro_url:
            productos_por_filtro[columna].append({"valor": valor, "productos": productos_en_filtro})
            print(f"🔹 Filtro: {columna} | Valor: {valor} | Productos: {len(productos_en_filtro)}")
            break

driver.quit()

# Scrapeo final usando list_scrap
list_scrap = []

driver = webdriver.Chrome(service=service, options=options)

for idx, product_url in enumerate(productos_urls):
    print(f"📦 Procesando producto {idx+1} de {len(productos_urls)}: {product_url}")
    driver.get(product_url)

    try:
        breadcrumb_elements = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element"))
        )
        if len(breadcrumb_elements) < 2 or breadcrumb_elements[1].text.strip() != "Maquillaje":
            print(f"⛔ Producto fuera de 'Maquillaje'. Saltando: {product_url}")
            continue
    except:
        print(f"⚠️ No se pudo obtener breadcrumb del producto: {product_url}")
        continue

    producto_info = {}
    producto_info["categoria"] = breadcrumb_elements[2].text.strip()
    producto_info["subcategoria"] = breadcrumb_elements[3].text.strip() if len(breadcrumb_elements) > 3 else np.nan

    producto_info["marca"] = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text

    titulo = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text
    nombre, descripcion = titulo.split(" - ") if " - " in titulo else (titulo, np.nan)
    producto_info["nombre"] = nombre
    producto_info["descripcion"] = descripcion

    try:
        precio = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text
        producto_info["precio"] = float(precio.replace(" €", "").replace(",", "."))
    except:
        producto_info["precio"] = np.nan

    try:
        n_val = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-number-review"))).get_attribute('innerHTML').strip().split(' ')[0]
        producto_info["numero_valoraciones"] = int(n_val)
    except:
        producto_info["numero_valoraciones"] = 0

    try:
        contenido = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        producto_info["valoracion"] = float(valoracion)
    except:
        producto_info["valoracion"] = 0

    variaciones = 0
    try:
        tonos = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        variaciones = int(tonos)
    except:
        pass
    try:
        tamanos = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        variaciones = int(tamanos)
    except:
        pass
    producto_info["num_variaciones"] = variaciones

    producto_info["fecha_extraccion"] = pd.to_datetime(date.today())

    for columna, filtros in productos_por_filtro.items():
        valores_detectados = []
        for filtro in filtros:
            if product_url in filtro["productos"]:
                valores_detectados.append(filtro["valor"])
        producto_info[columna] = ", ".join(valores_detectados) if valores_detectados else None

    list_scrap.append(producto_info)

driver.quit()

df = pd.DataFrame(list_scrap)
hoy = date.today().strftime("%Y-%m-%d")
nombre_archivo = f"productos_maquillaje_{hoy}.csv"
df.to_csv(nombre_archivo, index=False)
print(f"✅ CSV guardado como {nombre_archivo}")

Del scraping llamo a un datframe para hacer la insercion de los datos scrapeados. es decir, en la funcion del scrapeo tendra que haber un return del dataframe y al llamarla tendra que entrarle como parametro la url de sephora, y en la funcion de la primera insercion de datos tendra como parametro de entrada el dataframe obtenido del scrapeo

In [ ]:
# codigo completo de insercion y actualizacion de datos tras los scrapeos

import pandas as pd
import psycopg2
import numpy as np

# Conexión
conn = psycopg2.connect(
    dbname="prueba_sephora",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Leer CSV
df = pd.read_csv("productos_maquillaje.csv")
pd.set_option('display.max_columns', None)

for _, row in df.iterrows():
    nombre = row['nombre']
    descripcion = row['descripcion']
    marca = row['marca']
    categoria = row['categoria']
    subcategoria = row['subcategoria']
    precio = row['precio']
    numero_valoraciones = row['numero_valoraciones']
    num_variaciones = row['num_variaciones']
    valoracion = row['valoracion']
    fecha_extraccion = row['fecha_extraccion']
    
    efecto_sombra = row.get('efecto_sombra')
    textura = row.get('textura')

    # 1. ¿Existe el producto?
    cur.execute("SELECT id_producto FROM productos WHERE nombre = %s", (nombre,))
    producto_result = cur.fetchone()

    if producto_result:
        id_producto = producto_result[0]
        # Solo insertar histórico
        cur.execute("""
            INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones))
        print(f"Histórico actualizado para producto existente: {nombre}")
    else:
        # 2. Insertar Marca si no existe
        cur.execute("SELECT id_marca FROM marcas WHERE nombre_marca = %s", (marca,))
        marca_result = cur.fetchone()
        if marca_result:
            id_marca = marca_result[0]
        else:
            cur.execute("INSERT INTO marcas (nombre_marca) VALUES (%s) RETURNING id_marca", (marca,))
            id_marca = cur.fetchone()[0]

        # 3. Insertar Categoría si no existe
        cur.execute("SELECT id_categoria FROM categorias WHERE nombre_categoria = %s", (categoria,))
        categoria_result = cur.fetchone()
        if categoria_result:
            id_categoria = categoria_result[0]
        else:
            cur.execute("INSERT INTO categorias (nombre_categoria) VALUES (%s) RETURNING id_categoria", (categoria,))
            id_categoria = cur.fetchone()[0]

        # 4. Insertar Subcategoría si no existe
        cur.execute("SELECT id_subcategoria FROM subcategorias WHERE nombre_subcategoria = %s", (subcategoria,))
        subcategoria_result = cur.fetchone()
        if subcategoria_result:
            id_subcategoria = subcategoria_result[0]
        else:
            cur.execute("INSERT INTO subcategorias (nombre_subcategoria) VALUES (%s) RETURNING id_subcategoria", (subcategoria,))
            id_subcategoria = cur.fetchone()[0]

        # 5. Insertar Producto
        cur.execute("""
            INSERT INTO productos (nombre, descripcion, id_marca, id_categoria, id_subcategoria)
            VALUES (%s, %s, %s, %s, %s) RETURNING id_producto
        """, (nombre, descripcion, id_marca, id_categoria, id_subcategoria))
        id_producto = cur.fetchone()[0]

        # 6. Insertar Histórico
        cur.execute("""
            INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones))
        
        # 7. Insertar Filtros si existen
        # Efecto sombra
        if pd.notna(efecto_sombra):
            efectos = [e.strip() for e in efecto_sombra.split(",")]
            for efecto in efectos:
                cur.execute("SELECT id_efecto_sombra FROM efectos_sombra WHERE nombre_efecto = %s", (efecto,))
                efecto_result = cur.fetchone()
                if not efecto_result:
                    cur.execute("INSERT INTO efectos_sombra (nombre_efecto) VALUES (%s) RETURNING id_efecto_sombra", (efecto,))
                    id_efecto = cur.fetchone()[0]
                else:
                    id_efecto = efecto_result[0]
                cur.execute("INSERT INTO producto_efecto_sombra (id_producto, id_efecto_sombra) VALUES (%s, %s)", (id_producto, id_efecto))

        # Textura
        if pd.notna(textura):
            cur.execute("SELECT id_textura FROM texturas WHERE nombre_textura = %s", (textura,))
            textura_result = cur.fetchone()
            if not textura_result:
                cur.execute("INSERT INTO texturas (nombre_textura) VALUES (%s) RETURNING id_textura", (textura,))
                id_textura = cur.fetchone()[0]
            else:
                id_textura = textura_result[0]
            cur.execute("INSERT INTO producto_textura (id_producto, id_textura) VALUES (%s, %s)", (id_producto, id_textura))

        print(f"Producto nuevo insertado: {nombre}")

    conn.commit()

# Cerrar conexión
cur.close()
conn.close()
print("Proceso finalizado ✅")